# Timeseries classification from scratch

**Author:** [SteMula](https://github.com/SteMula/)<br>
**Last modified:** 2022/05/8<br>
**Description:** Training a timeseries classifier from scratch on the Bearings faul dataset from the Case Western Reserve Universiti archive.
https://engineering.case.edu/bearingdatacenter/download-data-file <br>

## Setup

In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

Note you have to download the file in local in order to import it as dataset.

97.mat is the dataset with the NORMAL SIGNAL

https://engineering.case.edu/sites/default/files/97.mat

105.mat is the dataset with the INNER RACE FAULT SIGNAL

https://engineering.case.edu/sites/default/files/105.mat

118.mat is the dataset with the BALL FAULT SIGNAL

https://engineering.case.edu/sites/default/files/118.mat

130.mat is the dataset with the OUTER RACE CENTERED FAULT SIGNAL

https://engineering.case.edu/sites/default/files/130.mat



In [ ]:
mat = scipy.io.loadmat('97.mat')
mat = mat['X097_DE_time']

mat_innerrace = scipy.io.loadmat('105.mat')
mat_innerrace = mat_innerrace['X105_DE_time']

mat_ball = scipy.io.loadmat('118.mat')
mat_ball = mat_ball['X118_DE_time']

mat_outer = scipy.io.loadmat('130.mat')
mat_outer = mat_outer['X130_DE_time']

print('normal', len(mat))
print('mat_innerrace', len(mat_innerrace))
print('mat_ball', len(mat_ball))

In the following function "split_data" I manipulate the input file I got before returning the data for training and validation

In [ ]:
def split_data(data, len_desired,class_number):
    # data is a numpy array of dimension (x,)
    # len_ desired is the size in which you want to split it, e.g. 500
    # example data is of dimension 243938 and you set len_desired of 500
    t_train = 0.67
    
    data_train = data[:int((len(data)*t_train))]
    data_test = data[int((len(data)*t_train)):]

    N_train = len(data_train)//len_desired
    x_train_ = data_train[:int((N_train*len_desired))]
    
    x_train_ = np.reshape(x_train_,(N_train,len_desired))
    y_train_ = class_number*np.ones(N_train)

    N_test = len(data_test)//len_desired

    x_test_ = data_test[:int((N_test*len_desired))]

    x_test_ = np.reshape(x_test_,(N_test,len_desired))
    y_test_ = class_number*np.ones(N_test)


    return x_train_, y_train_ , x_test_,y_test_

In [ ]:
x_train_mat, y_train_mat, x_test_mat, y_test_mat = split_data(np.squeeze(mat),500,0)
x_train_mat_ball, y_train_mat_ball, x_test_mat_ball, y_test_mat_ball = split_data(np.squeeze(mat_ball),500,1)
x_train_mat_innerrace, y_train_mat_innerrace, x_test_mat_innerrace, y_test_mat_innerrace= split_data(np.squeeze(mat_innerrace),500,2)
x_train_mat_outer, y_train_mat_outer, x_test_mat_outer, y_test_mat_outer= split_data(np.squeeze(mat_outer),500,3)



In [ ]:
x_train = np.concatenate((x_train_mat,x_train_mat_ball,x_train_mat_innerrace,x_train_mat_outer),axis=0)
y_train = np.concatenate((y_train_mat,y_train_mat_ball,y_train_mat_innerrace,y_train_mat_outer),axis=0)
x_test = np.concatenate((x_test_mat,x_test_mat_ball,x_test_mat_innerrace,x_test_mat_outer),axis=0)
y_test = np.concatenate((y_test_mat,y_test_mat_ball,y_test_mat_innerrace,y_test_mat_outer),axis=0)



## Visualize the data

Here we visualize one timeseries example for each class in the dataset.

In [ ]:
classes = np.unique(np.concatenate((y_train, y_test), axis=0))
print(classes)
plt.figure()
for c in range(0,2): #replace range(0,1) with classe
    c_x_train = x_train[y_train == c]
    plt.plot(c_x_train[0], label="class " + str(c))
plt.legend(loc="best")
plt.show()
plt.close()

In [ ]:
for c in range(2,4):
    c_x_train = x_train[y_train == c]
    plt.plot(c_x_train[0], label="class " + str(c))
plt.legend(loc="best")
plt.show()
plt.close()

## Standardize the data

Our timeseries are already in a single length (500). However, their values are
usually in various ranges. This is not ideal for a neural network;
in general we should seek to make the input values normalized.
For this specific dataset, the data is already z-normalized: each timeseries sample
has a mean equal to zero and a standard deviation equal to one. This type of
normalization is very common for timeseries classification problems, see
[Bagnall et al. (2016)](https://link.springer.com/article/10.1007/s10618-016-0483-9).

Note that the timeseries data used here are univariate, meaning we only have one channel
per timeseries example.
We will therefore transform the timeseries into a multivariate one with one channel
using a simple reshaping via numpy.
This will allow us to construct a model that is easily applicable to multivariate time
series.

In [ ]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

Finally, in order to use `sparse_categorical_crossentropy`, we will have to count
the number of classes beforehand.

In [ ]:
num_classes = len(np.unique(y_train))
print(num_classes)

Now we shuffle the training set because we will be using the `validation_split` option
later when training.

In [ ]:
idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

Standardize the labels to positive integers.
The expected labels will then be 0 and 1.

In [ ]:
# y_train[y_train == 0] = 0
# y_test[y_test == 0] = 0

# y_train[y_train == 2] = 1
# y_test[y_test == 2] = 1

# y_train[y_train == 4 ] = 2
# y_test[y_test == 4] = 2

## Build a model

We build a Fully Convolutional Neural Network originally proposed in
[this paper](https://arxiv.org/abs/1611.06455).
The implementation is based on the TF 2 version provided
[here](https://github.com/hfawaz/dl-4-tsc/).
The following hyperparameters (kernel_size, filters, the usage of BatchNorm) were found
via random search using [KerasTuner](https://github.com/keras-team/keras-tuner).

In [ ]:

def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)


model = make_model(input_shape=x_train.shape[1:])
keras.utils.plot_model(model, show_shapes=False)


# honestly I kept the shape of the net as it was done in the example since it gave me good result.
# Probably you can do better

## Train the model

In [ ]:
epochs = 500
batch_size = 64#32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=0,
)

## Evaluate model on test data

In [ ]:
model = keras.models.load_model("best_model.h5")

test_loss, test_acc = model.evaluate(x_test, y_test)

print("Test accuracy", test_acc)
print("Test loss", test_loss)
y_pred = model.predict(x_test)

In [ ]:
y_pred = y_pred.round(2)

y_test[205]
print(len(y_pred))

In [ ]:
xxx = np.zeros(len(y_pred))

arr = np.arange(num_classes)
for i in range(0,len(xxx)):
    xxx[i]= np.dot(arr,y_pred[i])

print(np.sum(abs(xxx-y_test))) #it should return 0 if all the classification are correct

In [ ]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()